In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import boto3
import botocore
from datetime import datetime

Get current parking sensor information

In [4]:
def get_live_parking():
    # find the parking dataset @ https://data.melbourne.vic.gov.au/Transport/On-street-Parking-Bay-Sensors/vh2v-4nfs
    parking_dataset_id = 'vh2v-4nfs'

    client = Socrata(
        "data.melbourne.vic.gov.au",
        "EC65cHicC3xqFXHHvAUICVXEr", # app token, just used to reduce throttling, not authentication
        timeout=120
    )

    api_results = client.get_all(parking_dataset_id)
    parking_sensors = pd.DataFrame.from_dict(api_results)
    parking_sensors = parking_sensors[['bay_id','st_marker_id','status','lat','lon']]
    parking_sensors = parking_sensors.astype({'lat':'float64', 'lon':'float64'})
    parking_sensors = parking_sensors.rename(columns={'st_marker_id':'marker_id'})

    parking_sensors = parking_sensors.drop_duplicates()
    parking_sensors['status'] = parking_sensors['status'].fillna('Unknown')
    parking_sensors['datetime'] = datetime.now()

    return parking_sensors

Get daily percentage from given dataframe

In [3]:
def get_daily_percentage_availability(df):
    df['DayOfWeek'] = df['datetime'].dt.day_of_week
    DailyParkingCounts = df.groupby('DayOfWeek').status.value_counts()
    DailyParkingCounts = DailyParkingCounts.unstack().reset_index()
    DailyParkingCounts['Percentage'] = (DailyParkingCounts['Unoccupied'] / (DailyParkingCounts['Unoccupied'] + DailyParkingCounts['Present']))
    DailyParkingCounts.reset_index(drop=True)
    return DailyParkingCounts[['DayOfWeek', 'Percentage']]

In [10]:
result = get_daily_percentage_availability(get_live_parking())

result.head()

status,DayOfWeek,Percentage
0,3,0.355916


In [37]:
df = pd.read_csv('../webapp/data/parkingsensor.csv')

df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='raise', utc=True)

In [38]:
df['datetime'].dt.hour

0          12
1          12
2          12
3          12
4          12
           ..
2624584     2
2624585     2
2624586     2
2624587     2
2624588     2
Name: datetime, Length: 2624589, dtype: int64

In [42]:
df = pd.DataFrame([datetime.now()])

series = pd.to_datetime(df[0], utc=True)
series.dt.hour

0    5
Name: 0, dtype: int64